# global ranking monitoring for LANEIGE (data from @COSME)

본 코드는 @COSME 사이트의 화장품 랭킹 정보를 추출하는 코드로 변수명을 일본어(홈페이지 원어) 기준으로 코드를 작성함.

## Import

In [1]:
import requests
import re
import time
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
import os

from tqdm import tqdm

## 데이터 수집 날짜 기록

랭킹은 매주 금요일에 업데이트 되기 때문에, 크롤링 로직이 `매주 금요일`에 실행되도록 자동화

In [ ]:
global collect_date
collect_date = datetime.today().strftime("%Y/%m/%d")
print("데이터 수집 날짜: ", collect_date)

데이터 수집 날짜:  2025/12/05


## COSME 카테고리별 랭킹 데이터 크롤링

### 카테고리 생성

| url 주소 기반 카테고리 코드 | 카테고리명 |
|-----------------------|----------|
| total | 종합뷰티랭킹|
| 800 | 종합스킨케어 |
| 1005 | 페이셜크림 |
| 904 | 페이셜마스크 |
| 803 | 베이스메이크업 |


<br>

\* _모니터링 대상 제품_

- 아토코스메(종합 뷰티 랭킹)
- 아토코스메(종합 스킨케어)
- 아토코스메(페이셜크림)
- 아토코스메(페이셜마스크)
- 아토코스메(베이스메이크업)

In [4]:
global category_name

category_name = {"total":"종합뷰티랭킹", 800: "종합스킨케어", 1005: "페이셜크림", 904: "페이셜마스크", 803: "베이스메이크업"}

### 종합 뷰티 랭킹

수집 대상 데이터

- `start_date ~ end_date` : 랭킹 기간(시작 날짜 ~ 종료 날짜)

- `rank` : 제품 순위

- `brand` : 브랜드명

- `product_name` : 제품명

- `release_date` : 제품 발매일

In [ ]:
def extract_cosme_laneige_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = []

    # 1. 랭킹 기간 추출
    date_block = soup.select_one("div#nav-rank-footer p")

    if date_block:
        text = date_block.text.strip()
        match = re.search(r"(\d{4}/\d{1,2}/\d{1,2})\s*[〜～-]\s*(\d{4}/\d{1,2}/\d{1,2})", text)
        start_date = match.group(1) if match else None
        end_date = match.group(2) if match else None
    else:
        start_date, end_date = None, None

    # 2. brand 필터 설정
    #brand_name = "LANEIGE(ラネージュ)"
    brand_name = "SHISEIDO" # test용

    # 3. 제품 정보 데이터 추출
    dl_list = soup.select("div#list-item dl")

    for dl in dl_list:

        # 3.1. 순위 추출
        rank_num_block = dl.select_one("span.rank-num")
        if not rank_num_block:
            continue

        img_tag = rank_num_block.select_one("img[alt]")

        # 상위 3위는 1위, 2위, 3위로 되어 있기 때문에 alt 속성에서 숫자 추출
        if img_tag:
            alt_text = img_tag.get("alt", "")
            numbers = re.findall(r"\d+", alt_text)
            rank = int(numbers[0]) if numbers else None

        # 4위 이하부터는 일반 텍스트로 되어 있기 때문에 텍스트에서 숫자 추출
        else:
            num_tag = rank_num_block.select_one("span.num")
            if not num_tag:
                continue
            numbers = re.findall(r"\d+", num_tag.text.strip())
            rank = int(numbers[0]) if numbers else None

        if rank is None:
            continue

        # 3.2. 브랜드명 추출
        brand_tag = dl.select_one("dd.summary span.brand a")
        if not brand_tag:
            continue

        brand = brand_tag.text.strip()
        if not brand_name in brand:
            continue

        # 3.3. 제품명 추출
        item_tag = dl.select_one("dd.summary span.item a")
        product_name = item_tag.text.strip() if item_tag else None

        # 3.4. 제품 발매일 추출
        release_tag = dl.select_one("p.onsale")
        if release_tag:
            release_text = release_tag.text.strip()
            match_release = re.search(r"発売日：(\d{4}/\d{1,2}/\d{1,2})", release_text)
            release_date = match_release.group(1) if match_release else None
        else:
            release_date = None

        # 추출한 데이터
        result.append({
            "collect_date": collect_date,
            "start_date": start_date,
            "end_date": end_date,
            "rank": rank,
            "brand": brand,
            "product_name": product_name,
            "release_date": release_date,
        })

    return result

# ---------------------------------------------------
def total_beauty_ranking():
    base_url = "https://www.cosme.net/ranking/products/page/{}"
    all_results = []

    for page in tqdm(range(1, 10)): # 종합 뷰티 랭킹은 1위 ~ 100위까지 (총 10페이지))
        url = (
            "https://www.cosme.net/ranking/products/"
            if page == 1
            else base_url.format(page)
        )

        res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if res.status_code != 200:
            print(f"페이지 로드 실패: {url}")
            continue

        page_results = extract_cosme_laneige_from_html(res.text)
        all_results.extend(page_results)

        time.sleep(1) # 서버 부하 방지

    return all_results

laneige_data = total_beauty_ranking()

100%|██████████| 9/9 [00:19<00:00,  2.12s/it]

[{'collect_date': '2025/12/05', 'start_date': '2025/11/27', 'end_date': '2025/12/3', 'rank': 38, 'brand': 'SHISEIDO', 'product_name': 'エッセンス スキンセッティング パウダー', 'release_date': '2025/9/1'}, {'collect_date': '2025/12/05', 'start_date': '2025/11/27', 'end_date': '2025/12/3', 'rank': 81, 'brand': 'SHISEIDO', 'product_name': 'エッセンス スキングロウ ファンデーション', 'release_date': '2023/9/1'}, {'collect_date': '2025/12/05', 'start_date': '2025/11/27', 'end_date': '2025/12/3', 'rank': 93, 'brand': 'SHISEIDO', 'product_name': 'オイデルミン エッセンスローション', 'release_date': '2023/3/1'}]


In [ ]:
# 종합 뷰티 랭킹 데이터 저장
data_total = pd.DataFrame(laneige_data)
data_total

### 종합 뷰티 랭킹 이외 카테고리

[종합 스킨케어, 페이셜크림, 페이셜마스크, 베이스메이크업] 에 해당하는 카테고리는 모두 동일한 페이지 형식을 가지고 있어서, 카테고리별 url만 구분하여 하나의 크롤링 코드를 활용함

수집 대상 데이터 (앞서 크롤링한 `종합 뷰티 랭킹` 수집 대상 데이터와 동일)

- `start_date ~ end_date` : 랭킹 기간(시작 날짜 ~ 종료 날짜)

- `rank` : 제품 순위

- `brand` : 브랜드명

- `product_name` : 제품명

- `release_date` : 제품 발매일

In [ ]:
# 동일 코드 반복 방지를 위한 함수화

def extract_cosme_laneige_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = []

    # 1. 랭킹 기간 추출
    date_block = soup.select_one("div#keyword-ranking-header p")

    if date_block:
        text = date_block.text.strip()
        match = re.search(r"(\d{4}/\d{1,2}/\d{1,2})\s*[〜～-]\s*(\d{4}/\d{1,2}/\d{1,2})", text)
        start_date = match.group(1) if match else None
        end_date = match.group(2) if match else None
    else:
        start_date, end_date = None, None

    # 2. brand 필터 설정
    #brand_name = "LANEIGE(ラネージュ)"
    brand_name = "SHISEIDO" # test용

    # 3. 제품 정보 데이터 추출
    dl_list = soup.select("div#keyword-ranking-list dl")

    for dl in dl_list:

        # 3.1. 순위 추출
        rank_num_block = dl.select_one("span.rank-num")
        if not rank_num_block:
            continue

        img_tag = rank_num_block.select_one("img[alt]")
        if img_tag:
            alt_text = img_tag.get("alt", "")
            numbers = re.findall(r"\d+", alt_text)
            rank = int(numbers[0]) if numbers else None
        else:
            num_tag = rank_num_block.select_one("span.num")
            if not num_tag:
                continue
            numbers = re.findall(r"\d+", num_tag.text.strip())
            rank = int(numbers[0]) if numbers else None

        if rank is None:
            continue

        # 3.2. 브랜드명 추출
        brand_tag = dl.select_one("dd.summary span.brand a")
        if not brand_tag:
            continue

        brand = brand_tag.text.strip()
        if not brand_name in brand:
            continue

        # 3.3. 제품명 추출
        item_tag = dl.select_one("dd.summary h4.item a")
        product_name = item_tag.text.strip() if item_tag else None

        # 3.4. 제품 발매일 추출
        release_tag = dl.select_one("p.onsale")
        if release_tag:
            release_text = release_tag.text.strip()
            match_release = re.search(r"発売日：(\d{4}/\d{1,2}/\d{1,2})", release_text)
            release_date = match_release.group(1) if match_release else None
        else:
            release_date = None

        # 추출한 데이터
        result.append({
            "collect_date": collect_date,
            "start_date": start_date,
            "end_date": end_date,
            "rank": rank,
            "brand": brand,
            "product_name": product_name,
            "release_date": release_date,
        })

    return result


# ---------------------------------------------------
# 카테고리 코드별로 크롤링

def category_ranking(category_id):
    base_url = f"https://www.cosme.net/categories/item/{category_id}/ranking/?page="
    all_results = []
    print(f"Data Crawling: category_id = {category_id}")
    for page in tqdm(range(1, 6)): # 랭킹은 1위 ~ 50위까지 (총 5페이지))
        url = (
            f"https://www.cosme.net/categories/item/{category_id}/ranking/"
            if page == 1
            else base_url+str(page)
        )

        res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if res.status_code != 200:
            print(f"페이지 로드 실패: {url}")
            continue

        page_results = extract_cosme_laneige_from_html(res.text)
        all_results.extend(page_results)

        time.sleep(1) # 서버 부하 방지

    return all_results


if __name__ == "__main__":
    category_ids = list(category_name.keys())[1:]
    collect_date = datetime.today().strftime("%Y/%m/%d")
    for category_id in category_ids:
        laneige_data = category_ranking(category_id)
        globals()[f'data_{category_id}'] = pd.DataFrame(laneige_data)

Data Crawling: category_id = 800


100%|██████████| 5/5 [00:14<00:00,  2.98s/it]


Category ID: 800, Data: [{'collect_date': '2025/12/05', 'start_date': '2025/9/4', 'end_date': '2025/12/3', 'rank': 21, 'brand': 'SHISEIDO', 'product_name': 'アルティミューン パワライジング セラム', 'release_date': '2025/3/1'}]
Data Crawling: category_id = 1005


100%|██████████| 5/5 [00:13<00:00,  2.64s/it]


Category ID: 1005, Data: [{'collect_date': '2025/12/05', 'start_date': '2025/9/4', 'end_date': '2025/12/3', 'rank': 47, 'brand': 'SHISEIDO', 'product_name': 'エッセンシャルイネルジャ ハイドレーティング クリーム', 'release_date': '2022/1/1'}]
Data Crawling: category_id = 904


100%|██████████| 5/5 [00:13<00:00,  2.63s/it]


Category ID: 904, Data: []
Data Crawling: category_id = 803


100%|██████████| 5/5 [00:16<00:00,  3.37s/it]

Category ID: 803, Data: [{'collect_date': '2025/12/05', 'start_date': '2025/9/4', 'end_date': '2025/12/3', 'rank': 3, 'brand': 'SHISEIDO', 'product_name': 'エッセンス スキンセッティング パウダー', 'release_date': '2025/9/1'}, {'collect_date': '2025/12/05', 'start_date': '2025/9/4', 'end_date': '2025/12/3', 'rank': 13, 'brand': 'SHISEIDO', 'product_name': 'エッセンス スキングロウ ファンデーション', 'release_date': '2023/9/1'}]


## 엑셀 파일에 데이터 저장

`COSME_LANEIGE_Ranking_History.xlsx` 파일에 카테고리별 데이터를 저장하거나, 이미 존재하는 경우 기존 시트 아래에 새로운 데이터를 추가함.

---

1. **파일 존재 여부 확인**
- 엑셀 파일이 없으면 새로 생성
- 이미 존재하면 기존 파일에 데이터를 추가

---

2. **(초기)파일 생성이 필요한 경우**
- ExcelWriter를 `mode='w'`로 열어 엑셀 파일을 생성합
- `category_name`에 포함된 각 카테고리에 대해:
  - 전역 변수(data_카테고리명)에서 크롤링한 데이터 가져옴
  - 데이터가 없거나 비어 있으면 지정된 columns로 빈 DataFrame 생성함
  - 해당 시트 이름으로 파일에 기록

---

3. **파일이 존재할 경우**
- ExcelWriter를 `mode='a'`로 열어 파일에 추가 기록함
- `if_sheet_exists='overlay'` 옵션으로 기존 시트를 유지
- `category_name`에 포함된 각 카테고리에 대해:
  - 전역 변수(data_카테고리명)에서 크롤링한 데이터 가져오고, 비어 있으면 skip
  - `header=False`로 설정하여 헤더 없이 기존 데이터에 이어서 추가



In [32]:
file_path = 'COSME_LANEIGE_Ranking_History.xlsx'
columns = ["collect_date", "start_date", "end_date", "rank", "brand", "product_name", "release_date"]

if not os.path.exists(file_path):

    with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        for name in category_name:
            df = globals().get(f'data_{name}', None)

            if df is None or len(df) == 0:
                df = pd.DataFrame(columns=columns)

            df.to_excel(writer, sheet_name=category_name[name], index=False)

else:

    with pd.ExcelWriter(
        file_path,
        engine='openpyxl',
        mode='a',
        if_sheet_exists='overlay'
    ) as writer:

        for name in category_name:
            sheet_name = category_name[name]
            df = globals().get(f'data_{name}', None)

            if df is None or len(df) == 0:
                continue

            start_row = writer.sheets[sheet_name].max_row

            df.to_excel(
                writer,
                sheet_name=sheet_name,
                startrow=start_row,
                index=False,
                header=False
            )


### 엑셀 파일로 저장한 크롤링 데이터 가져오기

Example>

앞선 크롤링 코드에서
1. Brand: Anua의 카테고리별 랭킹 내 데이터를 전부 불러오고 -> brand_name = "Anua"
2. Brand: SHISEIDO의 데이터를 전부 불러옴. -> brand_name = "SHISEIDO"

=> brand_name에 할당된 변수만 변경해서 데이터 불러옴.

**목적**: 원하는대로 크롤링 잘 해오는지 확인 + 엑셀 파일 새로 생성, 기존 엑셀 파일에 데이터 추가 확인을 위해

In [46]:
from IPython.display import display, HTML


for cate in category_name:
    data = pd.read_excel(file_path, sheet_name=category_name[cate])
    print("category name: ",category_name[cate])
    display(data)
    print("---------------------")

category name:  종합뷰티랭킹


,collect_date,start_date,end_date,rank,brand,product_name,release_date
0,2025/12/05,2025/11/27,2025/12/3,35,Anua,PDRN ヒアルロン酸100 セラム,2024/12/16
1,2025/12/05,2025/11/27,2025/12/3,42,Anua,PDRN100ヒアルロン酸セラムマスク,2025/5/2
2,2025/12/05,2025/11/27,2025/12/3,38,SHISEIDO,エッセンス スキンセッティング パウダー,2025/9/1
3,2025/12/05,2025/11/27,2025/12/3,81,SHISEIDO,エッセンス スキングロウ ファンデーション,2023/9/1
4,2025/12/05,2025/11/27,2025/12/3,93,SHISEIDO,オイデルミン エッセンスローション,2023/3/1


---------------------
category name:  종합스킨케어


,collect_date,start_date,end_date,rank,brand,product_name,release_date
0,2025/12/05,2025/9/4,2025/12/3,3,Anua,PDRN100ヒアルロン酸セラムマスク,2025/5/2
1,2025/12/05,2025/9/4,2025/12/3,12,Anua,PDRN ヒアルロン酸100 セラム,2024/12/16
2,2025/12/05,2025/9/4,2025/12/3,21,SHISEIDO,アルティミューン パワライジング セラム,2025/3/1


---------------------
category name:  페이셜크림


...

---------------------
category name:  페이셜마스크


,collect_date,start_date,end_date,rank,brand,product_name,release_date
0,2025/12/05,2025/9/4,2025/12/3,1,Anua,PDRN100ヒアルロン酸セラムマスク,2025/5/2
1,2025/12/05,2025/9/4,2025/12/3,13,Anua,アゼライン酸15インテンスカーミングセラムマスク,2025/1/13
2,2025/12/05,2025/9/4,2025/12/3,18,Anua,ドクダミ77スージングトナーシカエキソソームマスク,2024/5/27
3,2025/12/05,2025/9/4,2025/12/3,21,Anua,桃70ナイアシンセラムマスク,2023/11/6
4,2025/12/05,2025/9/4,2025/12/3,26,Anua,レチノール0.3ナイアシンナイトケアマスク,2025/7/29
5,2025/12/05,2025/9/4,2025/12/3,41,Anua,ライス70 発酵保湿マスクパック,2025/1/13
6,2025/12/05,2025/9/4,2025/12/3,47,Anua,ナイアシン10アルブチンセラムマスク,2025/4/14
7,2025/12/05,2025/9/4,2025/12/3,49,Anua,PDRNヒアルロン酸セラムデイリーマスク,2025/8/30


---------------------
category name:  베이스메이크업


,collect_date,start_date,end_date,rank,brand,product_name,release_date
0,2025/12/05,2025/9/4,2025/12/3,3,SHISEIDO,エッセンス スキンセッティング パウダー,2025/9/1
1,2025/12/05,2025/9/4,2025/12/3,13,SHISEIDO,エッセンス スキングロウ ファンデーション,2023/9/1


---------------------
